# Aula6 - Doc2Query

[Unicamp - IA368DD: Deep Learning aplicado a sistemas de busca.](https://www.cpg.feec.unicamp.br/cpg/lista/caderno_horario_show.php?id=1779)

Autor: Marcus Vinícius Borela de Castro

[Repositório no github](https://github.com/marcusborela/deep_learning_em_buscas_unicamp)

Stage: calculating metrics after expanding texts with queries 

# Organizando o ambiente

In [8]:
import pickle

In [23]:
import json

In [1]:
import pandas as pd

In [2]:
DIRETORIO_TRABALHO = '/home/borela/fontes/deep_learning_em_buscas_unicamp/local/doc2query'

In [3]:
assert os.path.exists(DIRETORIO_TRABALHO), f"Path para {DIRETORIO_TRABALHO} não existe!"

In [4]:
import os

export JVM_PATH=/usr/lib/jvm/java-11-openjdk-amd64/lib/server/libjvm.so
export JAVA_HOME=/usr/lib/jvm/java-11-openjdk-amd64

In [5]:
os.environ['JVM_PATH'] = '/usr/lib/jvm/java-11-openjdk-amd64/lib/server/libjvm.so'
os.environ['JAVA_HOME'] = '/usr/lib/jvm/java-11-openjdk-amd64'

In [6]:
from pyserini.search.lucene import LuceneSearcher

/home/borela/miniconda3/envs/treinapython39/lib/python3.9/site-packages/tqdm/auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


# Carregando os dados de documentos expandidos

In [9]:
with open(f"{DIRETORIO_TRABALHO}/trec-covid-corpus-sem-texto.pickle", "rb") as f:
  corpus_sem_texto = pickle.load(f)

In [10]:
with open(f"{DIRETORIO_TRABALHO}/trec-covid-corpus-com-texto.pickle", "rb") as f:
  corpus_com_texto = pickle.load(f)

In [11]:
type(corpus_com_texto), corpus_com_texto[0]

(list,
 {'_id': 'ug7v899j',
  'title': 'Clinical features of culture-proven Mycoplasma pneumoniae infections at King Abdulaziz University Hospital, Jeddah, Saudi Arabia',
  'text': 'OBJECTIVE: This retrospective chart review describes the epidemiology and clinical features of 40 patients with culture-proven Mycoplasma pneumoniae infections at King Abdulaziz University Hospital, Jeddah, Saudi Arabia. METHODS: Patients with positive M. pneumoniae cultures from respiratory specimens from January 1997 through December 1998 were identified through the Microbiology records. Charts of patients were reviewed. RESULTS: 40 patients were identified, 33 (82.5%) of whom required admission. Most infections (92.5%) were community-acquired. The infection affected all age groups but was most common in infants (32.5%) and pre-school children (22.5%). It occurred year-round but was most common in the fall (35%) and spring (30%). More than three-quarters of patients (77.5%) had comorbidities. Twenty-four 

In [118]:
%%time
lista_carga = []
for doc in corpus_com_texto:
    lista_carga.append({"id": doc['_id'], "contents":f"Title: {doc['title']}\nText:{doc['text']}\n{doc['texto_gerado']}"})

CPU times: user 141 ms, sys: 90.3 ms, total: 232 ms
Wall time: 231 ms


Se desejar carregar os documentos sem texto mas expandidos, rodar comando abaixo:

In [ ]:
%%time
for doc in corpus_sem_texto:
    lista_carga.append({"id": doc['_id'], "contents":f"Title: {doc['title']}\nText:{doc['texto_gerado']}"})

CPU times: user 24.8 ms, sys: 3.98 ms, total: 28.8 ms
Wall time: 28.4 ms


Caso contrário, se desejar carregar os documentos sem texto, rodar comando abaixo: (se quiser, pode não carregá-los)

In [82]:
%%time
for doc in corpus_sem_texto:
    lista_carga.append({"id": doc['_id'], "contents":f"Title: {doc['title']}"})

CPU times: user 11.6 ms, sys: 8.2 ms, total: 19.8 ms
Wall time: 19.5 ms


Visualizando os dados

In [29]:
print(lista_carga[0]['contents'])

Title: Clinical features of culture-proven Mycoplasma pneumoniae infections at King Abdulaziz University Hospital, Jeddah, Saudi Arabia
Text:OBJECTIVE: This retrospective chart review describes the epidemiology and clinical features of 40 patients with culture-proven Mycoplasma pneumoniae infections at King Abdulaziz University Hospital, Jeddah, Saudi Arabia. METHODS: Patients with positive M. pneumoniae cultures from respiratory specimens from January 1997 through December 1998 were identified through the Microbiology records. Charts of patients were reviewed. RESULTS: 40 patients were identified, 33 (82.5%) of whom required admission. Most infections (92.5%) were community-acquired. The infection affected all age groups but was most common in infants (32.5%) and pre-school children (22.5%). It occurred year-round but was most common in the fall (35%) and spring (30%). More than three-quarters of patients (77.5%) had comorbidities. Twenty-four isolates (60%) were associated with pneum

In [30]:
print(lista_carga[-1]['contents'])

Title: Vascular Life during the COVID-19 Pandemic Reminds Us to Prepare for the Unexpected
Text:what was unexpected life like novod and lacrimal tragedy; what was unexpected life like no ivd; what is the life expectancy for the covd 19 pandemic avianity; what was unexpected complications during the covd; what was unexpected heart like during the covid; what was unexpected life like during the covid; what was unexpected life like at rosa life; what was unexpected life like during the covd; what was unexpected baggage life; what was unexpected life like at the covid


# Construindo o índice

In [119]:

# Escreve a lista de dicionários sem a chave 'title' em um arquivo JSONL
with open(f"{DIRETORIO_TRABALHO}/trec-covid-expanded/trec-covid-corpus-expanded.jsonl", 'w') as arquivo:
    for doc in lista_carga:
        # Converte o dicionário em uma string JSON
        linha_json = json.dumps(doc)
        # Escreve a linha JSON no arquivo
        arquivo.write(f'{linha_json}\n')

In [120]:
%%time
!python -m pyserini.index.lucene \
  --collection JsonCollection \
  --input {DIRETORIO_TRABALHO}/trec-covid-expanded/ \
  --index indexes/trec_covid_expanded \
  --generator DefaultLuceneDocumentGenerator \
  --threads 9 \
  --storePositions --storeDocvectors --storeRaw

2023-04-11 15:23:46,319 INFO  [main] index.IndexCollection (IndexCollection.java:380) - Setting log level to INFO
2023-04-11 15:23:46,320 INFO  [main] index.IndexCollection (IndexCollection.java:383) - Starting indexer...
2023-04-11 15:23:46,320 INFO  [main] index.IndexCollection (IndexCollection.java:384) - ============ Loading Parameters ============
2023-04-11 15:23:46,320 INFO  [main] index.IndexCollection (IndexCollection.java:385) - DocumentCollection path: /home/borela/fontes/deep_learning_em_buscas_unicamp/local/doc2query/trec-covid-expanded/
2023-04-11 15:23:46,320 INFO  [main] index.IndexCollection (IndexCollection.java:386) - CollectionClass: JsonCollection
2023-04-11 15:23:46,320 INFO  [main] index.IndexCollection (IndexCollection.java:387) - Generator: DefaultLuceneDocumentGenerator
2023-04-11 15:23:46,320 INFO  [main] index.IndexCollection (IndexCollection.java:388) - Threads: 9
2023-04-11 15:23:46,320 INFO  [main] index.IndexCollection (IndexCollection.java:389) - Langua

In [121]:
!mv indexes/trec_covid_expanded {DIRETORIO_TRABALHO}/indexes

In [122]:
!ls {DIRETORIO_TRABALHO}/indexes

trec_covid_expanded
trec_covid_expanded_com_e_sem_texto
trec_covid_expanded_full_so_texto_expandido


In [123]:
caminho_indice_trec_covid_com_expansao = f'{DIRETORIO_TRABALHO}/indexes/trec_covid_expanded'

# Baixando os dados e preparando para avaliação 

## Queries

Se precisar carregar a partir de url

if not os.path.exists(f'{DIRETORIO_TRABALHO}/queries.jsonl.gz'):
    !wget https://huggingface.co/datasets/BeIR/trec-covid/resolve/main/queries.jsonl.gz
    !mv queries.jsonl.gz {DIRETORIO_TRABALHO}/


Nome do arquivo compactado
arquivo_gz =  f'{DIRETORIO_TRABALHO}/queries.jsonl.gz'

import gzip
Descompacte o arquivo para a memória
with gzip.open(arquivo_gz, 'rt') as f:
    # Leia o conteúdo do arquivo descompactado
    queries = [json.loads(line) for line in f]

Exiba os dados carregados
print(f"{type(queries)} len(queries): {len(queries)}")    


In [45]:
del queries

{'_id': '1',
 'text': 'what is the origin of COVID-19',
 'metadata': {'query': 'coronavirus origin',
  'narrative': "seeking range of information about the SARS-CoV-2 virus's origin, including its evolution, animal source, and first transmission into humans"}}

In [46]:
from pyserini.search import get_topics

In [47]:
topics = get_topics('covid-round5')
print(f'{len(topics)} queries total')

50 queries total


In [49]:
topics[50]

{'question': 'what is known about an mRNA vaccine for the SARS-CoV-2 virus?',
 'query': 'mRNA vaccine coronavirus',
 'narrative': 'Looking for studies specifically focusing on mRNA vaccines for COVID-19, including how mRNA vaccines work, why they are promising, and any results from actual clinical studies.'}

## qrel de teste

In [51]:
if not os.path.exists(f'{DIRETORIO_TRABALHO}/test.tsv'):
    !wget https://huggingface.co/datasets/BeIR/trec-covid-qrels/raw/main/test.tsv
    !mv test.tsv {DIRETORIO_TRABALHO}/

In [111]:
qrel = pd.read_csv(f"{DIRETORIO_TRABALHO}/test.tsv", sep="\t", header=None, 
                   skiprows=1, names=["query", "docid", "rel"])

In [112]:
qrel.head()

,query,docid,rel
0,1,005b2j4b,2
1,1,00fmeepz,1
2,1,g7dhmyyo,2
3,1,0194oljo,1
4,1,021q9884,1


In [54]:
from tqdm import tqdm

In [90]:
# Run all queries in topics, retrive top 1k for each query
def run_all_queries(file, topics, searcher, num_max_hits=100):
  """
  A função run_all_queries é responsável por realizar todas as consultas armazenadas no dicionário topics utilizando o objeto searcher fornecido e salvar os resultados em um arquivo de texto.
  Usada no notebook da aula 2

  Parâmetros:

  file: caminho do arquivo de saída onde serão salvos os resultados das consultas.
  topics: dicionário contendo as consultas a serem executadas. Cada consulta é representada por uma chave única no dicionário. O valor correspondente a cada chave é um outro dicionário contendo as informações da consulta, como seu título e outras informações relevantes.
  searcher: objeto do tipo Searcher que será utilizado para realizar as consultas.
  num_max_hits: número máximo de documentos relevantes que serão retornados para cada consulta.
  Retorno:

  A função não retorna nenhum valor, mas salva os resultados das consultas no arquivo especificado em file.
  Comentário:

  A função usa a biblioteca tqdm para exibir uma barra de progresso enquanto executa as consultas.
  O número de consultas concluídas é impresso a cada 100 consultas.
  """
  print(f'Running {len(topics)} queries in total')
  with open(file, 'w') as runfile:
    cnt = 0
    for id in tqdm(topics, desc='Running Queries'):
        # print(f'id = {id}')
        query = topics[id]['question']
        # print(f'query = {query}')

        hits = searcher.search(query, num_max_hits)
        for i in range(0, len(hits)):
            _ = runfile.write(f'{id} Q0 {hits[i].docid} {i+1} {hits[i].score:.6f} ComExpansao\n')
            # = runfile.write('{} Q0 {} {} {:.6f} Pyserini\n'.format(id, hits[i].docid, i+1, hits[i].score))
        cnt += 1
        if cnt % 100 == 0:
            print(f'{cnt} queries completed')


In [124]:
searcher = LuceneSearcher(caminho_indice_trec_covid_com_expansao)

In [125]:
searcher.set_bm25(k1=0.82, b=0.68)  

In [126]:
path_run = f"{DIRETORIO_TRABALHO}/runs"
path_run_com_expansao = path_run + '/run-trec-covid-expanded-bm25.txt'

In [127]:
%%time
if not os.path.exists(path_run):
  os.makedirs(path_run)
  print('pasta criada!')
else:
  print('pasta já existia!')


pasta já existia!
CPU times: user 44 µs, sys: 8 µs, total: 52 µs
Wall time: 103 µs


In [128]:
num_max_hits = 1000

In [129]:
%%time
run_all_queries(path_run_com_expansao, topics, searcher, num_max_hits)

Running 50 queries in total


Running Queries: 100%|██████████| 50/50 [00:02<00:00, 24.27it/s]

CPU times: user 2.16 s, sys: 61.9 ms, total: 2.22 s
Wall time: 2.06 s


In [130]:
assert os.path.exists(path_run_com_expansao), f"Pasta {path_run_com_expansao} não criada!"

In [131]:
!head {path_run_com_expansao}

44 Q0 xfjexm5b 1 12.792300 ComExpansao
44 Q0 ihyj9fpi 2 11.962600 ComExpansao
44 Q0 d6w8fu3b 3 11.815900 ComExpansao
44 Q0 zwyueevh 4 11.574700 ComExpansao
44 Q0 5cm6122n 5 11.571500 ComExpansao
44 Q0 2zkesall 6 11.436500 ComExpansao
44 Q0 28utunid 7 11.353400 ComExpansao
44 Q0 qi1henyy 8 11.329500 ComExpansao
44 Q0 b2yalnrx 9 11.231800 ComExpansao
44 Q0 cenc2u15 10 11.217900 ComExpansao


# Calculando métricas

In [132]:
run = pd.read_csv(f"{path_run_com_expansao}", sep="\s+", 
                  names=["query", "q0", "docid", "rank", "score", "system"])

In [133]:
run.head()

,query,q0,docid,rank,score,system
0,44,Q0,xfjexm5b,1,12.7923,ComExpansao
1,44,Q0,ihyj9fpi,2,11.9626,ComExpansao
2,44,Q0,d6w8fu3b,3,11.8159,ComExpansao
3,44,Q0,zwyueevh,4,11.5747,ComExpansao
4,44,Q0,5cm6122n,5,11.5715,ComExpansao


In [134]:
run = run.to_dict(orient="list")

In [135]:
run['query'][0], run['docid'][0], run['rank'][0]

(44, 'xfjexm5b', 1)

In [136]:
run.keys()

dict_keys(['query', 'q0', 'docid', 'rank', 'score', 'system'])

In [107]:
from evaluate import load

In [137]:
trec_eval = load("trec_eval")

In [138]:
qrel.head()

,query,docid,rel,q0
0,1,005b2j4b,2,q0
1,1,00fmeepz,1,q0
2,1,g7dhmyyo,2,q0
3,1,0194oljo,1,q0
4,1,021q9884,1,q0


In [139]:
qrel["q0"] = "q0"
qrel_dict = qrel.to_dict(orient="list")

In [140]:
qrel_dict['query'][0], qrel_dict['docid'][0], qrel_dict['rel'][0]

(1, '005b2j4b', 2)

In [141]:
results = trec_eval.compute(predictions=[run], references=[qrel_dict])

Resultados:

        ndcg@10: 66,4% com todos os documentos (mas com expansão apenas nos com texto)
        ndcg@10: 64,39% com apenas os documentos com texto expandido no índice
        ndcg@10: 61,5% com todos os documentos expandidos (seja do texto, seja do title)

ndcg@10: 66,4% com todos os documentos (mas com expansão apenas nos com texto)

In [ ]:
results

{'runid': 'ComExpansao',
 'num_ret': 50000,
 'num_rel': 24673,
 'num_rel_ret': 9994,
 'num_q': 50,
 'map': 0.19839799878460124,
 'gm_map': 0.13903258902331478,
 'bpref': 0.34394252858331353,
 'Rprec': 0.29468011493155,
 'recip_rank': 0.8966666666666667,
 'P@5': 0.708,
 'P@10': 0.664,
 'P@15': 0.6493333333333335,
 'P@20': 0.6369999999999999,
 'P@30': 0.6073333333333333,
 'P@100': 0.48760000000000003,
 'P@200': 0.40149999999999997,
 'P@500': 0.28624,
 'P@1000': 0.19988,
 'NDCG@5': 0.6633977345833243,
 'NDCG@10': 0.6150074260089573,
 'NDCG@15': 0.5988840533590788,
 'NDCG@20': 0.5821469114168832,
 'NDCG@30': 0.5552191954757941,
 'NDCG@100': 0.45805893820459004,
 'NDCG@200': 0.3945725313299633,
 'NDCG@500': 0.3711874011972914,
 'NDCG@1000': 0.42390065206749156}

ndcg@10: 64,39% com apenas os documentos com texto expandido no índice

In [142]:
results

{'runid': 'ComExpansao',
 'num_ret': 50000,
 'num_rel': 24673,
 'num_rel_ret': 9772,
 'num_q': 50,
 'map': 0.19273438064173107,
 'gm_map': 0.1387019248126997,
 'bpref': 0.33477383683640133,
 'Rprec': 0.29335170022334883,
 'recip_rank': 0.9066666666666667,
 'P@5': 0.72,
 'P@10': 0.674,
 'P@15': 0.6439999999999999,
 'P@20': 0.632,
 'P@30': 0.604,
 'P@100': 0.494,
 'P@200': 0.40449999999999997,
 'P@500': 0.28256,
 'P@1000': 0.19544,
 'NDCG@5': 0.673303285278764,
 'NDCG@10': 0.624451187564097,
 'NDCG@15': 0.5945089612849035,
 'NDCG@20': 0.5793756183962986,
 'NDCG@30': 0.5523891856158754,
 'NDCG@100': 0.46256019710155855,
 'NDCG@200': 0.39741418129986095,
 'NDCG@500': 0.366410494754734,
 'NDCG@1000': 0.41412060873701345}

ndcg@10: 61,5% com todos os documentos (com e sem texto)

In [ ]:
results

{'runid': 'ComExpansao',
 'num_ret': 50000,
 'num_rel': 24673,
 'num_rel_ret': 10086,
 'num_q': 50,
 'map': 0.20161615245732417,
 'gm_map': 0.14030842775589514,
 'bpref': 0.3475260256531407,
 'Rprec': 0.29795822037524966,
 'recip_rank': 0.8858333333333335,
 'P@5': 0.7120000000000001,
 'P@10': 0.66,
 'P@15': 0.636,
 'P@20': 0.628,
 'P@30': 0.5933333333333333,
 'P@100': 0.49080000000000007,
 'P@200': 0.40519999999999995,
 'P@500': 0.28804,
 'P@1000': 0.20172000000000004,
 'NDCG@5': 0.665883536449219,
 'NDCG@10': 0.6154165482614988,
 'NDCG@15': 0.5924465108102263,
 'NDCG@20': 0.5779014712934913,
 'NDCG@30': 0.5460710909772548,
 'NDCG@100': 0.46035578744806105,
 'NDCG@200': 0.39789330917345794,
 'NDCG@500': 0.3739604199086901,
 'NDCG@1000': 0.42708093410037584}